#### I could not save trained model
#### So please to check working of this notebook train it on 5 or 10 sentences

In [ ]:
import json
import numpy as np
import pandas as pd
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('train_data1.json', 'r',encoding="utf8") as json_file:
    json_data = json.load(json_file)


data_keys = json_data.keys()


In [ ]:

for Languages_name in data_keys:

            list_of_train_pairs = []

            for language_pair, language_data in json_data.items():
                if(language_pair == f"{Languages_name}"):
                  for data_type, data_entries in language_data.items():
                      print(f"  Data Type: {data_type}")
                      print(f"langauages: {language_pair}")
                      for entry_id, entry_data in data_entries.items():
                          source_sentence = entry_data["source"]
                          target_sentence = entry_data["target"]
                          list_of_train_pairs.append([source_sentence,target_sentence])

            # list_of_train_pairs = random.sample(list_of_train_pairs, 5000)

            list_of_train_pairs = list_of_train_pairs[0:5]

            list_of_train_pairs


            with open('test_data1_final.json', 'r',encoding="utf8") as json_file:
                json_data_val = json.load(json_file)


            entity_id_val = []
            list_of_val_source = []

            # Process JSON data
            for language_pair, language_data in json_data_val.items():
                if(language_pair == f"{Languages_name}"):
                   for data_type, data_entries in language_data.items():
                       print(f"  Data Type: {data_type}")
                       for entry_id, entry_data in data_entries.items():
                          source = entry_data["source"]
                          list_of_val_source.append([entry_id,source])


            list_of_val_source = list_of_val_source[:5]



            max_length = 0
            max_sentence = ""

            for i in range(len(list_of_train_pairs)):
                current_length = len(list_of_train_pairs[i][0].split(' '))
                if current_length > max_length:
                    max_length = current_length
                    max_sentence = list_of_train_pairs[i][0]


            MAX_LENGTH_SOURCE =  max_length + 1


            # Initialize variables to keep track of the maximum length and the corresponding sentence
            max_length = 0
            max_sentence = ""

            for i in range(len(list_of_train_pairs)):
                # Calculate the length of the current sentence
                current_length = len(list_of_train_pairs[i][1].split(' '))

                # Check if the current sentence is longer than the previous maximum
                if current_length > max_length:
                    max_length = current_length
                    max_sentence = list_of_train_pairs[i][1]

            MAX_LENGTH_TGT =  max_length + 1


            Start_of_sentence_token = 0
            End_of_sentence_token = 1

            class Language_formate:
                def __init__(self):
                    self.word2index = {}
                    self.word2count = {}
                    self.index2word = {0: "SOS", 1: "EOS"}
                    self.n_words = 2  # Count SOS and EOS

                def add_sentence(self, sentence):
                    for word in sentence.split(' '):
                        self.add_word(word)

                def add_word(self, word):
                    if word not in self.word2index:
                        self.word2index[word] = self.n_words
                        self.word2count[word] = 1
                        self.index2word[self.n_words] = word
                        self.n_words += 1
                    else:
                        self.word2count[word] += 1


            input_lang = Language_formate()
            output_lang = Language_formate()


            for x in list_of_train_pairs:
                    input_lang.add_sentence(x[0])
                    output_lang.add_sentence(x[1])


            # def indexesFromSentence(lang, sentence):
            #     return [lang.word2index[word] for word in sentence.split(' ')]
            def indexesFromSentence(lang, sentence):
                try:
                    return [lang.word2index[word] for word in sentence.split(' ')]
                except KeyError:
                    return [0]


            def tensorFromSentence(lang, sentence):
                indexes = indexesFromSentence(lang, sentence)
                indexes.append(End_of_sentence_token)
                return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)



            def load_data(batch_size,input_lang, output_lang, list_of_train_pairs):
                n = len(list_of_train_pairs)


                input_ids = np.zeros((n, MAX_LENGTH_SOURCE), dtype=np.int32)
                print(input_ids.shape)
                target_ids = np.zeros((n, MAX_LENGTH_TGT), dtype=np.int32)

                for idx, (inp, tgt) in enumerate(list_of_train_pairs):
                    inp_ids = indexesFromSentence(input_lang, inp)
                    tgt_ids = indexesFromSentence(output_lang, tgt)
                    inp_ids.append(End_of_sentence_token)
                    tgt_ids.append(End_of_sentence_token)
                    input_ids[idx, :len(inp_ids)] = inp_ids
                    target_ids[idx, :len(tgt_ids)] = tgt_ids

                train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                                           torch.LongTensor(target_ids).to(device))

                train_sampler = RandomSampler(train_data)
                train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
                return input_lang, output_lang, train_dataloader

            class EncoderLSTM(nn.Module):
                def __init__(self, input_size, hidden_size, dropout_p=0.1):
                    super(EncoderLSTM, self).__init__()
                    self.hidden_size = hidden_size

                    self.embedding = nn.Embedding(input_size, hidden_size)
                    self.lstm = nn.LSTM(hidden_size, hidden_size,batch_first=True)
                    self.dropout = nn.Dropout(dropout_p)

                def forward(self, input):
                    embedded = self.dropout(self.embedding(input))
                    output, (hidden, cell) = self.lstm(embedded)
                    return output, (hidden, cell)

            class DecoderGRU(nn.Module):
                def __init__(self, hidden_size, output_size):
                    super(DecoderGRU, self).__init__()
                    self.embedding = nn.Embedding(output_size, hidden_size)
                    self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
                    self.out = nn.Linear(hidden_size, output_size)

                def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
                    batch_size = encoder_outputs.size(0)
                    decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(Start_of_sentence_token)

                    # Initialize hx as a tuple with two None elements
                    # hx = (None, None)
                    hx = encoder_hidden
                    # print(hx)

                    decoder_outputs = []

                    for i in range(MAX_LENGTH_TGT):
                        decoder_output, hx = self.forward_step(decoder_input, hx)
                        decoder_outputs.append(decoder_output)

                        if target_tensor is not None:
                                        # Teacher forcing: Feed the target as the next input
                                        decoder_input = target_tensor[:, i].unsqueeze(1)  # Teacher forcing
                        else:
                            # Without teacher forcing: use its own predictions as the next input
                            _, topi = decoder_output.topk(1)
                            decoder_input = topi.squeeze(-1).detach()  # detach from history as input


                    decoder_outputs = torch.cat(decoder_outputs, dim=1)
                    decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
                    return decoder_outputs, hx, None

                def forward_step(self, input, hx):
                    output = self.embedding(input)
                    output = F.relu(output)

                    # Ensure hx is a tuple, even if cell state is not used
                    # if not isinstance(hx, tuple):
                    #     hx = (hx, None)

                    output, hx = self.gru(output, hx)
                    # print(hx)
                    output = self.out(output)
                    return output, hx


            def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
                      decoder_optimizer, criterion):

                total_loss = 0
                for data in dataloader:
                    input_tensor, target_tensor = data

                    encoder_optimizer.zero_grad()
                    decoder_optimizer.zero_grad()

                    encoder_outputs, (encoder_hidden, cell_state) = encoder(input_tensor)
                    decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

                    loss = criterion(
                        decoder_outputs.view(-1, decoder_outputs.size(-1)),
                        target_tensor.view(-1)
                    )
                    loss.backward()

                    encoder_optimizer.step()
                    decoder_optimizer.step()

                    total_loss += loss.item()

                return total_loss / len(dataloader)

            import time
            import math

            def asMinutes(s):
                m = math.floor(s / 60)
                s -= m * 60
                return '%dm %ds' % (m, s)

            def timeSince(since, percent):
                now = time.time()
                s = now - since
                es = s / (percent)
                rs = es - s
                return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

            def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
                           print_every=100):
                start = time.time()
                print_loss_total = 0  # Reset every print_every

                encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
                decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
                criterion = nn.NLLLoss()

                for epoch in range(1, n_epochs + 1):
                    loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
                    print_loss_total += loss

                    if epoch % print_every == 0:
                        print_loss_avg = print_loss_total / print_every
                        print_loss_total = 0
                        print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                                    epoch, epoch / n_epochs * 100, print_loss_avg))




            def evaluate(encoder, decoder, sentence, input_lang, output_lang):
                with torch.no_grad():
                    input_tensor = tensorFromSentence(input_lang, sentence)

                    encoder_outputs, (encoder_hidden,hidden_cell) = encoder(input_tensor)
                    decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

                    _, topi = decoder_outputs.topk(1)
                    decoded_ids = topi.squeeze()
                    # print(decoded_ids)

                    decoded_words = []
                    for idx in decoded_ids:
                        if idx.item() == End_of_sentence_token:
                            decoded_words.append('<EOS>')
                            break
                        decoded_words.append(output_lang.index2word[idx.item()])
                    # print(decoded_words)
                return decoded_words

            def evaluateRandomly(encoder, decoder, n=10):
                data_write = []
                for i in range(len(list_of_val_source)):
                    pair = list_of_val_source
                    output_words = evaluate(encoder, decoder, pair[i][1], input_lang, output_lang)
                    result = " "
                    result_sen = result.join(output_words)
                    output_sentence = pair[i][0]+" "+'"'+result_sen+'"'
                    result_string =output_sentence.replace("<EOS>", "" )
                    data_write.append(result_string)

                import csv

                # Create or open a CSV file in write mode
                csv_file_name = f"{Languages_name}.csv"
                with open(csv_file_name, mode="w", newline="", encoding="utf-8") as csv_file:
                   # for x in data_write:
                   #     csv_writer = csv.writer(csv_file)
                   #     csv_writer.writerow([x])
                    csv_writer = csv.writer(csv_file)
                    for sentence in data_write:
                            # Ensure there are no extra spaces by stripping the sentence
                            cleaned_sentence = sentence.strip()
                            # Split the sentence into words and write them as a list
                            word_list = cleaned_sentence.split()
                            csv_file.write(cleaned_sentence + '\n')





            hidden_size = 32
            batch_size = 32

            input_lang, output_lang, train_dataloader = load_data(batch_size,input_lang,output_lang,list_of_train_pairs)


            encoder = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
            decoder = DecoderGRU(hidden_size, output_lang.n_words).to(device)

            train(train_dataloader, encoder, decoder, 80, print_every=5)

            # import pickle
            # with open(f"{Languages_name}_enc",'wb') as f:
            #     pickle.dump(encoder,f)

            # with open(f"{Languages_name}_dec",'wb') as f:
            #      pickle.dump(decoder,f)


            encoder.eval()
            decoder.eval()
            evaluateRandomly(encoder, decoder, list_of_val_source)


  Data Type: Train
langauages: English-Bengali
  Data Type: Test
(5, 23)
0m 0s (- 0m 2s) (5 6%) 4.0744
0m 0s (- 0m 2s) (10 12%) 3.8720
0m 0s (- 0m 2s) (15 18%) 3.6622
0m 0s (- 0m 2s) (20 25%) 3.4467
0m 0s (- 0m 1s) (25 31%) 3.2333
0m 1s (- 0m 1s) (30 37%) 3.0330
0m 1s (- 0m 1s) (35 43%) 2.8559
0m 1s (- 0m 1s) (40 50%) 2.7096
0m 1s (- 0m 1s) (45 56%) 2.5930
0m 1s (- 0m 0s) (50 62%) 2.4976
0m 1s (- 0m 0s) (55 68%) 2.4132
0m 1s (- 0m 0s) (60 75%) 2.3363
0m 2s (- 0m 0s) (65 81%) 2.2699
0m 2s (- 0m 0s) (70 87%) 2.2081
0m 2s (- 0m 0s) (75 93%) 2.1501
0m 2s (- 0m 0s) (80 100%) 2.0950
  Data Type: Train
langauages: English-Gujarati
  Data Type: Test
(5, 26)
0m 0s (- 0m 2s) (5 6%) 4.0378
0m 0s (- 0m 1s) (10 12%) 3.8719
0m 0s (- 0m 1s) (15 18%) 3.7049
0m 0s (- 0m 1s) (20 25%) 3.5364
0m 0s (- 0m 1s) (25 31%) 3.3669
0m 0s (- 0m 1s) (30 37%) 3.2011
0m 0s (- 0m 1s) (35 43%) 3.0492
0m 1s (- 0m 1s) (40 50%) 2.9206
0m 1s (- 0m 0s) (45 56%) 2.8153
0m 1s (- 0m 0s) (50 62%) 2.7235
0m 1s (- 0m 0s) (55 68%)

In [ ]:
import os
import pandas as pd

records = []
for file in os.listdir("."):
    # print(file)

    if file.split(".")[0] == "answer":
        continue

    if file.split(".")[-1] == "csv":
        with open(f"./{file}", encoding='utf-8') as f:
            record = f.readlines()
            print(f"{file}   {len(record)}")
            records.extend(record)
            # records.extend(f.readlines())

# records[:100]

len(records)

English-Bengali.csv   5
English-Gujarati.csv   5
English-Hindi.csv   5
English-Kannada.csv   5
English-Malayalam.csv   5
English-Tamil.csv   5
English-Telgu.csv   5


35

In [ ]:
with open("./answer.csv", "w+", encoding="utf-8") as file:
    # for ele in records:
    file.writelines(records)
print("anser.csv have ",len(records),"Entries")

anser.csv have  35 Entries
